In [63]:
using Plots
using FFTW
using LinearAlgebra

&#x1F4D1; Note: the `operations.jl` source code helps writting mathematical symbols:
For example:
- `Fᴵ` replaces the `ifft` function (inverse Fourier Transform)
- the convolution sign `⊗` is used rather than the `conv` function
- the reverse sign `⦰` is also used to replace the `reverse` function
- the concatenation sign `⧺` is used to make a vertical concatenation operation
- the transpose function `(.)ᵀ` is use to transpose a vector or a matrix 

In [64]:
include("../modules/operations.jl");

In [65]:
using LaTeXStrings

In [66]:
using DSP

##### &#x0030;&#xFE0F;&#x20E3;  Implementing the Receiver from Lab1

In [67]:
𝑓ₛ = 61.44 * 1e6; @show 𝑓ₛ # samples/s
Nᵨ = 1e-2; @show Nᵨ # typed \N\_rho for ratio to the sampling frequency in samples/s
𝑁τ = Nᵨ * 𝑓ₛ ; @show 𝑁τ; # Periodicity 
𝐶ₚ = 144; # Cyclic Prefix

𝑓ₛ = 6.144e7
Nᵨ = 0.01
𝑁τ = 614400.0


In [68]:
Nₒₛ = 2048; @show Nₒₛ;  # OFDM Symbol Size
𝑁ₚₛₛ = Nₒₛ + 𝐶ₚ; @show 𝑁ₚₛₛ; # Duration = PSS Signal + Cyclic Prefix samples

Nₒₛ = 2048
𝑁ₚₛₛ = 2192


In [69]:
## Receive the signal
include("../data/julia/rxsignal_withchannelandfreqoff.jl");

In [76]:
typeof(rxs3)

Matrix{ComplexF64} (alias for Array{Complex{Float64}, 2})

In [77]:
vec(rxs3)[1:2]

2-element Vector{ComplexF64}:
  -7.30420513781688 + 19.17827614654979im
 -6.235568689840052 - 16.03725969281452im

In [90]:
# complex_f32_matrix = [ComplexF32(round(Int, real(c)), round(Int, imag(c))) for c in rxs3]
complex_f32_matrix = [ComplexF32(real(c), imag(c)) for c in rxs3]
typeof(complex_f32_matrix)
vec(complex_f32_matrix)
# sᵣₓ³ = MtlArray(vec(complex_i64_matrix)); # RX Received Signal 3 File Handle

616447-element Vector{ComplexF32}:
  -7.304205f0 + 19.178276f0im
 -6.2355685f0 - 16.03726f0im
  -2.258558f0 + 2.895619f0im
  -7.150135f0 - 4.681072f0im
 -5.9986644f0 - 2.1136062f0im
  1.9124045f0 + 11.702587f0im
 -27.349476f0 + 2.8440108f0im
  21.337273f0 - 14.893436f0im
 -27.127567f0 - 0.23518242f0im
 -7.3059773f0 + 9.734681f0im
  -3.887132f0 + 9.80836f0im
  -3.943259f0 + 1.2724535f0im
  -4.371416f0 - 9.179027f0im
              ⋮
  7.4728937f0 + 2.9326513f0im
 -3.9892952f0 + 3.2345488f0im
   1.087239f0 + 4.362068f0im
 -3.4475145f0 + 11.3241f0im
  3.3832786f0 - 0.13001971f0im
 -5.8508615f0 - 8.86564f0im
   3.193117f0 + 2.1947856f0im
  -6.394119f0 - 1.3345447f0im
   9.183092f0 - 5.8471622f0im
  1.9831575f0 + 11.299379f0im
 -13.366416f0 - 2.1861851f0im
  4.0488343f0 + 10.654216f0im

In [91]:
## Assign the received signal a variable sᵣₓ³
sᵣₓ³ = MtlArray(vec(complex_f32_matrix)); # RX Received Signal 3 File Handle
@show size(sᵣₓ³), typeof(sᵣₓ³);

(size(sᵣₓ³), typeof(sᵣₓ³)) = ((616447,), MtlVector{ComplexF32, Metal.MTL.MTLResourceStorageModePrivate})


In [92]:
## Load the template signal
include("../data/julia/pss2.jl");

In [84]:
using Metal

In [94]:
pss_2[1:2]

2-element Vector{Complex{Int64}}:
 -363 + 0im
 -363 + 0im

In [93]:
pss_complex_f32_matrix = [ComplexF32(real(c), imag(c)) for c in pss_2]
typeof(pss_complex_f32_matrix)
vec(pss_complex_f32_matrix)


2048-element Vector{ComplexF32}:
 -363.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
          ⋮
 -363.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im
  362.0f0 + 0.0f0im
  362.0f0 + 0.0f0im

In [100]:
# allocate + initialize
pss_complex_f32_matrix
s̃ₚₛₛ² = MtlArray(vec(pss_complex_f32_matrix)); @show typeof(s̃ₚₛₛ²);
s̃ₚₛₛ²[1:2]
# s̃ₚₛₛ² = MtlArray(pss_2); @show typeof(s̃ₚₛₛ²);

typeof(s̃ₚₛₛ²) = MtlVector{ComplexF32, Metal.MTL.MTLResourceStorageModePrivate}


2-element MtlVector{ComplexF32, Metal.MTL.MTLResourceStorageModePrivate}:
 -363.0f0 + 0.0f0im
 -363.0f0 + 0.0f0im

In [99]:
## Assign the reference signal to a variable s̃ₚₛₛ²
## convert the signal in time domain

s̃ₚₛₛ² = 𝓕⁻¹(s̃ₚₛₛ²);   # (inverse fourier transform) in time domain typed s\tilde
# s̃ₚₛₛ² ./= norm(s̃ₚₛₛ²);  @show size(s̃ₚₛₛ²), 𝐶ₚ-1    # Normalize
# s̃ₚₛₛ² = s̃ₚₛₛ²[(end-(𝐶ₚ-1)):end] ⧺ s̃ₚₛₛ² # align verticaly end is 2048 FFT size
# @show size(s̃ₚₛₛ²), typeof(s̃ₚₛₛ²); 

LoadError: ArgumentError: cannot take the CPU address of a MtlVector{ComplexF32, Metal.MTL.MTLResourceStorageModePrivate}

In [87]:
## Prepare the reference signal for convolution
s̃ₚₛₛ²ˣ = (⦰(s̃ₚₛₛ²))ˣ; # reverse conjugate in one operation otherwise can lose peak
@show length(s̃ₚₛₛ²ˣ), typeof(s̃ₚₛₛ²ˣ);

(length(s̃ₚₛₛ²ˣ), typeof(s̃ₚₛₛ²ˣ)) = (2192, Vector{ComplexF64})


In [88]:
## Perform the convolution between the 2 signals
rᵣₓ³ = 10 * log10.(abs.( sᵣₓ³ ⊗ s̃ₚₛₛ²ˣ ))
@show size( rᵣₓ³ ), typeof(rᵣₓ³); 

LoadError: Metal does not support Float64 values, try using Float32 instead

# &#x1F4DA; References

- [ ] [How can I define the frequency resolution in FFT? And what is the difference on interpreting the results between high and low frequency resolution?](https://www.researchgate.net/post/How-can-I-define-the-frequency-resolution-in-FFT-And-what-is-the-difference-on-interpreting-the-results-between-high-and-low-frequency-resolution)